In [2]:
import sys
sys.path.append('../python_packages_static/')
import pandas as pd
import pyemu
import os, glob
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import numpy as np

# After Running prior Monte Carlo, we can postprocess to perform rejection sampling and prepare for next run

In [3]:
run_ensemble=False

In [4]:
if run_ensemble==True:
    wkdir = '../run_data/' #data in which PEST++ runs take place and results live
else:
    wkdir = '../output/parallel/'
pstroot='prior_mc'

### read in the ensemble of observations

In [5]:
ens = pd.read_csv(os.path.join(wkdir,'{}.0.obs.csv'.format(pstroot)), index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '../output/parallel/prior_mc.0.obs.csv'

### read in the phi values for the ensemble

In [ ]:
phi = pd.read_csv(os.path.join(wkdir,'{}.phi.actual.csv'.format(pstroot)))

### read in the parameter ensemble

In [ ]:
parens = pd.read_csv(os.path.join(wkdir, '{}.0.par.csv'.format(pstroot)), index_col=0)

In [ ]:
parens.head()

### look at a histogram of Phi and make a subjective call for where to truncate for rejection sampling

In [ ]:
phi.T.dropna().iloc[6:].hist(bins=50)

In [ ]:
phi

In [ ]:
# adjust threshold based on histogram
phi_threshold_for_keeping = 35000 # prior_mc
allphi=phi.T.dropna().iloc[6:].copy()
allphi.columns=['phi']
keepreals = allphi.loc[allphi.phi<phi_threshold_for_keeping].index
keepreals

### now trim phi to only the realizations to keep

In [ ]:
allphi.loc[keepreals].hist(bins=50)

In [ ]:
# truncate the ensemble to only keepreals (based on phi)
ens = ens.loc[keepreals]
# set percent_discrepancy to absolute value because we don't care about the sign
ens.perc_disc = ens.perc_disc.apply(lambda x: np.abs(x))

In [ ]:
# further truncate to eliminate bad mass balance runs 
print(len(ens))
ens = ens.loc[ens.perc_disc<0.01]
ens

In [ ]:
# reset keepreals based now both on phi and mass balance
keepreals = ens.index.values

In [ ]:
# look at phi again
allphi.loc[keepreals].hist(bins=50)

In [ ]:
allphi.loc[keepreals]

In [ ]:
pst = pyemu.Pst(os.path.join(wkdir,'{}.pst'.format(pstroot)))

In [ ]:
obs = pst.observation_data

In [ ]:
obs

In [ ]:
ens.to_csv(f'../notebooks_report/final_obs_ensembles/{pstroot}.prior.obs.ens.csv')

### can plot out histograms for all observations into a large, multipage PDF file

In [ ]:
with PdfPages(os.path.join(wkdir, 'postproc', '{}_obs_hist_plots.pdf'.format(pstroot))) as outpdf:
    for cob in obs.obsnme.values:
        plt.figure()
        ens[cob].hist(bins=50)
        obsval = obs.loc[cob].obsval
        plt.axvline(obsval, color='orange')
        plt.title(cob)
        outpdf.savefig()
        plt.close('all')